In [146]:
from math import pi
import pandas as pd

from bokeh.io import show, output_notebook
from bokeh.models import LinearColorMapper, BasicTicker, PrintfTickFormatter, ColorBar
from bokeh.plotting import figure
from bokeh.sampledata.unemployment1948 import data

output_notebook()

data['Year'] = data['Year'].astype(str)
data = data.set_index('Year')
data.drop('Annual', axis=1, inplace=True)
data.columns.name = 'Month'

years = list(data.index)
months = list(data.columns)

# reshape to 1D array or rates with a month and year for each row.
df = pd.DataFrame(data.stack(), columns=['rate']).reset_index()

# this is the colormap from the original NYTimes plot
colors = ["#75968f", "#a5bab7", "#c9d9d3", "#e2e2e2", "#dfccce", "#ddb7b1", "#cc7878", "#933b41", "#550b1d"]
mapper = LinearColorMapper(palette=colors, low=df.rate.min(), high=df.rate.max())

TOOLS = "hover,save,pan,box_zoom,reset,wheel_zoom"

p = figure(title="US Unemployment ({0} - {1})".format(years[0], years[-1]),
           x_range=years, y_range=list(reversed(months)),
           x_axis_location="above", plot_width=900, plot_height=400,
           tools=TOOLS, toolbar_location='below',
           tooltips=[('date', '@Month @Year'), ('rate', '@rate%')])

p.grid.grid_line_color = None
p.axis.axis_line_color = None
p.axis.major_tick_line_color = None
p.axis.major_label_text_font_size = "5pt"
p.axis.major_label_standoff = 0
p.xaxis.major_label_orientation = pi / 2

p.rect(x="Year", y="Month", width=1, height=1,
       source=df,
       fill_color={'field': 'rate', 'transform': mapper},
       line_color=None)

color_bar = ColorBar(color_mapper=mapper, major_label_text_font_size="5pt",
                     ticker=BasicTicker(desired_num_ticks=len(colors)),
                     formatter=PrintfTickFormatter(format="%d%%"),
                     label_standoff=6, border_line_color=None, location=(0, 0))
p.add_layout(color_bar, 'right')

show(p)      # show the plot

Loading BokehJS ...

In [105]:
import os; os.chdir('/Users/Play/Code/AI/master-thesis/src/')
import sklearn.metrics
import numpy as np
import helpers as hp
import constants as ct

In [150]:
df = hp.read_smth_results(ct.RUN_DIR / '2019-03-01@14-08-56@lrcn@smth@dummy'/ 'results.json')
df = results[['template_id', 'top1_conf']]
id2temp = hp.read_smth_id2labels(ct.SMTH_LABELS2ID)

conf_mat = sklearn.metrics.confusion_matrix(results['template_id'].values, results['top1_conf'].values)
conf_mat = conf_mat.astype(np.float) / conf_mat.astype(np.float).sum(axis=1)[:, np.newaxis]

stacked = pd.DataFrame(pd.DataFrame(conf_mat).stack(), columns=['Overlap']).reset_index()
stacked.columns = ['Ground Truth', 'Prediction', 'Overlap']
stacked = stacked.join(id2temp, on='Ground Truth').drop(labels=['Ground Truth', 'id'], axis=1).rename(columns={'template':'Ground Truth'})
stacked = stacked.join(id2temp, on='Prediction').drop(labels=['Prediction', 'id'], axis=1).rename(columns={'template':'Prediction'})

TOOLS = "hover,save,pan,box_zoom,reset,wheel_zoom"
p = figure(title="Confusion Matrix",
           x_range=stacked['Ground Truth'].unique().tolist(), y_range=list(reversed(stacked['Ground Truth'].unique().tolist())),
           x_axis_location="above", plot_width=800, plot_height=800,
           tools=TOOLS, toolbar_location='below', tooltips=[('Overlap', '@Overlap{0.0000}'),
                                                            ('Ground Truth', '@{Ground Truth}'),
                                                            ('Prediction', '@{Prediction}'),
                                                           ])
p.grid.grid_line_color = None
p.axis.axis_line_color = None
p.axis.major_tick_line_color = None
p.axis.major_label_text_font_size = "8pt"
p.axis.major_label_standoff = 0
p.xaxis.major_label_orientation = pi / 2.5

mapper = LinearColorMapper(palette='Greys256', low=0, high=1)
p.rect(x="Prediction", y="Ground Truth", width=1, height=1,
       source=stacked,
       fill_color={'field': 'Overlap', 'transform': mapper},
       line_color=None)
show(p)